ImageNet에서 훈련한 컨브넷을 가지고 시작하겠습니다. 케라스에는 이렇게 사용할 수 있는 컨브넷이 많습니다. VGG16, VGG19, Xception, ResNet50 등입니다. 이 중에 어느 것을 사용해도 딥드림을 구현할 수 있습니다. 당연히 어떤 컨브넷을 선택했느냐에 따라 시각화에 영향을 미칩니다. 각 컨브넷 구조가 학습한 특성이 다르기 때문입니다. 원래 딥드림에서 사용한 컨브넷은 인셉션 모델입니다. 실제로 인셉션이 멋진 딥드림 이미지를 잘 만듭니다. 여기에서도 케라스의 인셉션 V3 모델을 사용하겠습니다.

In [1]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

그 다음 손실을 계산합니다. 경사 상승법으로 최대화할 값이다. 5장 필터 시각화에서 특정 층의 필터 값을 최대화하였다. 여기서는 여러 층에  있는 모든 필터 활성화를 동시에 최대화한다. 특별히 상위 층에 있는 활성화의 L2 노름에 대한 가중치 합을 최대화하겠다. 정확히 어떤 층들을 선택했는지에 따라 만들어 내는 시각 요소에 큰 영향을 미친다. 어떤 층을 선택할지 파리미터로 손쉽게 바꿀 수 있어야 좋다. 하위 층은 기하학적인 패턴을 만들고 상위 층을 ImageHet에 있는 클래스로 보이는 시각 요소를 만든다. 먼저 임의로 4개의 층을 선택해 보겠다. 나중에 다른 설정을 다양하게 시도해본느 것이 좋다.

In [4]:
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 0.3,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

이제 손실 텐서를 정의하겠다. 선택한 층의 활성화에 대한 L2 노름의 가중치 합이다.

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

그 다음 경사 상승법 과정을 준비한다.

In [6]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_value = outs[1]
    return loss_value, grad_value

def gradinet_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print("... loss", i ,loss_value)
        x += step * grad_values
    return x

마지막으로 먼저 이미지를 처리하기 위한 스케일 리스트를 정의한다. 스케일은 이전 스케일보다 1.4배 크다. 작은 이미지로 시작해서 점점 크기를 키운다.

![deep dream process](https://s3.amazonaws.com/book.keras.io/img/ch8/deepdream_process.png)

가장 작은 것에서 가장 큰 스케일까지 연속적인 각 단계에서 정의한 손실이 최대화되도록 경사 상승법을 수행한다. 경사 상승법이 실행된 후 이미지 크기를 40% 증가시킨다.

스케일을 연속적으로 증가시키멵서 이미지 상세를 많이 잃지 않도록 간닿나 기교를 사용한다. 스케일을 늘린 후 이미지 손실된 디테일을 재주입한다. 원본 이미지가 크기를 늘렸을 대 어땠는지 알기 때문에 가능하다. 작은 미지 크기 S와 큰 이미지 크기 L이 주어지면 크기 L로 변경된 원본 이미지와 크기 S로 변경된 원본 이미지 사이의 창이를 계산한다. 이 차이가 S에서 L로 변경되었을 때 잃어버린 디테일이다.

In [9]:
import numpy as np
import scipy
from keras.preprocessing import image

이 코드는 다음에 나오는 유틸리티 함수를 사용한다. 넘파이 배열 기반의 함수이며 이름으로 역할을 알 수 있다.

In [13]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order = 1)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [14]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/original_photo_deep_dream.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                   for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print("prpcess img size ", shape)
    img = resize_img(img, shape)
    img = gradinet_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    same_size_original = resize_img(shrunk_original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='./datasets/final_dream.png')

prpcess img size  (178, 178)
... loss 0 0.28384238
... loss 1 0.42410055
... loss 2 0.5940663
... loss 3 0.78658235
... loss 4 0.9741764
... loss 5 1.1833122
... loss 6 1.3849878
... loss 7 1.5884495
... loss 8 1.7742459
... loss 9 2.005937
... loss 10 2.2309623
... loss 11 2.4984365
... loss 12 2.7062707
... loss 13 2.8973129
... loss 14 3.1220598
... loss 15 3.2961917
... loss 16 3.5387912
... loss 17 3.7714255
... loss 18 3.9536703
... loss 19 4.136682
prpcess img size  (250, 250)
... loss 0 0.73781586
... loss 1 1.2348982
... loss 2 1.7068805
... loss 3 2.1089332
... loss 4 2.4481187
... loss 5 2.7978797
... loss 6 3.1147895
... loss 7 3.457151
... loss 8 3.7299998
... loss 9 3.997414
... loss 10 4.2418356
... loss 11 4.555108
... loss 12 4.8153973
... loss 13 5.0955963
... loss 14 5.3482714
... loss 15 5.62992
... loss 16 5.8174276
... loss 17 6.0101624
... loss 18 6.1992517
... loss 19 6.4060125
prpcess img size  (350, 350)
... loss 0 1.0287725
... loss 1 1.6691835
... loss 2 2.2

In [16]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/mountain-4021090_640.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                   for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print("prpcess img size ", shape)
    img = resize_img(img, shape)
    img = gradinet_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    same_size_original = resize_img(shrunk_original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='./datasets/final_dream_test.png')

prpcess img size  (228, 326)
... loss 0 0.6473802
... loss 1 0.85713863
... loss 2 1.1688275
... loss 3 1.456235
... loss 4 1.7661134
... loss 5 2.0499036
... loss 6 2.321937
... loss 7 2.6157699
... loss 8 2.87372
... loss 9 3.157087
... loss 10 3.4065619
... loss 11 3.652048
... loss 12 3.9259062
... loss 13 4.186429
... loss 14 4.4473486
... loss 15 4.6854377
... loss 16 4.9372654
... loss 17 5.158122
... loss 18 5.365385
... loss 19 5.600559
prpcess img size  (320, 457)
... loss 0 1.1147645
... loss 1 1.840356
... loss 2 2.4591699
... loss 3 2.9944005
... loss 4 3.439838
... loss 5 3.8455524
... loss 6 4.2185664
... loss 7 4.5759535
... loss 8 4.913655
... loss 9 5.2432375
... loss 10 5.5602245
... loss 11 5.8384533
... loss 12 6.133918
... loss 13 6.406173
... loss 14 6.6646748
... loss 15 6.9402456
... loss 16 7.1741743
... loss 17 7.3978186
... loss 18 7.681485
... loss 19 7.914432
prpcess img size  (448, 640)
... loss 0 1.3453637
... loss 1 2.1399431
... loss 2 2.7939863
... lo

![](./datasets/final_dream_test.png)

In [17]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/netherlands-2153807_640.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                   for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print("prpcess img size ", shape)
    img = resize_img(img, shape)
    img = gradinet_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    same_size_original = resize_img(shrunk_original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='./datasets/final_dream_test2.png')

prpcess img size  (218, 326)
... loss 0 0.6279238
... loss 1 0.80984926
... loss 2 1.0587766
... loss 3 1.3172957
... loss 4 1.5823405
... loss 5 1.8184133
... loss 6 2.1034167
... loss 7 2.3314066
... loss 8 2.5887413
... loss 9 2.814781
... loss 10 3.0667849
... loss 11 3.2815437
... loss 12 3.5223575
... loss 13 3.7490566
... loss 14 3.9808078
... loss 15 4.1913915
... loss 16 4.3962355
... loss 17 4.597286
... loss 18 4.8014154
... loss 19 4.9703684
prpcess img size  (306, 457)
... loss 0 1.1724377
... loss 1 1.7742028
... loss 2 2.2877333
... loss 3 2.750134
... loss 4 3.1502333
... loss 5 3.5217285
... loss 6 3.8542686
... loss 7 4.19529
... loss 8 4.5251665
... loss 9 4.8362513
... loss 10 5.103448
... loss 11 5.375659
... loss 12 5.635949
... loss 13 5.8991075
... loss 14 6.1379747
... loss 15 6.387535
... loss 16 6.6347804
... loss 17 6.875309
... loss 18 7.0928125
... loss 19 7.3205366
prpcess img size  (429, 640)
... loss 0 1.4063393
... loss 1 2.095735
... loss 2 2.693293
.

![](./datasets/final_dream_test2.png)

In [20]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/colored-pencils-4031668_640.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                   for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print("prpcess img size ", shape)
    img = resize_img(img, shape)
    img = gradinet_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    same_size_original = resize_img(shrunk_original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='./datasets/final_dream_test3.png')

prpcess img size  (217, 326)
... loss 0 0.81265914
... loss 1 1.0170896
... loss 2 1.2315736
... loss 3 1.4434633
... loss 4 1.6782501
... loss 5 1.9209416
... loss 6 2.1434317
... loss 7 2.3752732
... loss 8 2.6013143
... loss 9 2.8325388
... loss 10 3.0613365
... loss 11 3.2755532
... loss 12 3.501019
... loss 13 3.70096
... loss 14 3.9167626
... loss 15 4.140281
... loss 16 4.3166027
... loss 17 4.5190387
... loss 18 4.720596
... loss 19 4.9067087
prpcess img size  (305, 457)
... loss 0 1.1617348
... loss 1 1.720062
... loss 2 2.2127364
... loss 3 2.6547081
... loss 4 3.0975118
... loss 5 3.5066652
... loss 6 3.9254045
... loss 7 4.3418465
... loss 8 4.701977
... loss 9 5.0962296
... loss 10 5.4288564
... loss 11 5.7696013
... loss 12 6.07978
... loss 13 6.4098125
... loss 14 6.7254696
... loss 15 7.035905
... loss 16 7.34121
... loss 17 7.6281147
... loss 18 7.930848
... loss 19 8.193346
prpcess img size  (427, 640)
... loss 0 1.3787445
... loss 1 2.1163688
... loss 2 2.747806
... 

![](./datasets/final_dream_test3.png)